### Challenge 35: Implement DH with negotiated groups, and break with malicious "g" parameters

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

<div class="alert alert-block alert-info">

Implement DH with negotiated groups, and break with malicious "g" parameters

```A->B```    Send "p", "g"
    
```B->A```    Send ACK
    
```A->B```    Send "A"
    
```B->A```    Send "B"
    
```->B```     Send ```AES-CBC(SHA1(s)[0:16], iv=random(16), msg) + iv```
    
```B->A```    Send ```AES-CBC(SHA1(s)[0:16], iv=random(16), A's msg) + iv```

Do the MITM attack again, but play with "g". What happens with:

```g = 1```
    
```g = p```
    
```g = p - 1```
    
Write attacks for each.

<div class="alert alert-block alert-warning">

#### **When does this ever happen?**

Honestly, not that often in real-world systems. If you can mess with "g", chances are you can mess with something worse. Most systems pre-agree on a static DH group. But the same construction exists in Elliptic Curve Diffie-Hellman, and this becomes more relevant there.

</div>

</div>

In [1]:
import numpy as np
import cryptopals as cp
import sha1
from Crypto import Random

<div class="alert alert-block alert-info">

Implement DH with negotiated groups, and break with malicious "g" parameters
    
</div>

---
From this challenge description, it's not clear to me if we can somehow manipulate Alice's g parameter as well as Bob's?

---

<div class="alert alert-block alert-info">

Use the code you just worked out to build a protocol and an "echo" bot. You don't actually have to do the network part of this if you don't want; just simulate that. The protocol is:

```A->B```    Send "p", "g"
    
```B->A```    Send ACK
    
```A->B```    Send "A"
    
```B->A```    Send "B"
    
```->B```     Send ```AES-CBC(SHA1(s)[0:16], iv=random(16), msg) + iv```
    
```B->A```    Send ```AES-CBC(SHA1(s)[0:16], iv=random(16), A's msg) + iv```
    
</div>

<div class="alert alert-block alert-info">
    
```A->B```    Send "p", "g"

```B->A```    Send ACK
    
</div>

In [2]:
p = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff
g = 2

<div class="alert alert-block alert-info">
    
```A->B```    Send "A"
</div>

In [3]:
# Alice's private key is 'a'
a = np.random.randint(0, 2**31) % p

# Alice's public key is 'A'
A = pow(g, a, p)

---
**This is where Mallory / Eve gets involved!**

---

<div class="alert alert-block alert-info">
    
```B->A```    Send "B"
   
</div>

<div class="alert alert-block alert-info">

Do the MITM attack again, but play with "g". What happens with:

**```g = 1```**
    
```g = p```
    
```g = p - 1```
    
</div>

Bob computes his own public key ```B``` using the parameters (```p``` and ```g```) supplied by Alice and sends ```B``` back to her.  I'm not implementing the transfer portion of the protocol...just pretend.

In [4]:
g_evil = 1

# Bob's private key is 'b'
b = np.random.randint(0, 2**31) % p

# Bob's public key is 'B'.  He computed this using g_evil -- and shares it with Alice.
B = pow(g_evil, b, p)

---
By setting ```g=1```, Mallory has forced Bob's public key to be a power of 1 ==> 

**B is always 1.**

Bob computes a shared secret using Alice's public key ```A```, his private key ```b```, and the public modulus ```p```

---
I'm stuck (maybe) here.  If Alice computed her public key using the good / not evil ```g```, and Mallory doesn't tamper with that, Bob would use her true public key based on the original ```g``` to calculate the shared secret.  On the other hand, Alice will use Bob's evil public key (B = 1) to compute what she thinks is the shared secret.  Thus, sB will always be a power of 1, or 1 mod p.  So...the shared secrets that are hashed to produce the AES key don't match.  

I think this means Mallory / Eve would be able to intercept and decrypt messages from Alice --> Bob, but not vice versa.  Furthermore, I don't think Alice would be able to decrypt Bob's messages and Bob would not be able to decrypt Alice's.?

**What am I missing?**

Note:  If Mallory was able to tamper with ```g``` before Alice used it, they both will compute a shared secret = 1 and this becomes a viable MITM attack.

<div class="alert alert-block alert-info">

Do the MITM attack again, but play with "g". What happens with:

```g = 1```
    
**```g = p```**
    
```g = p - 1```
    
</div>

In [25]:
p = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff
g = 2

In [26]:
# Alice's private key is 'a'
a = np.random.randint(0, 2**31) % p

# Alice's public key is 'A'
A = pow(g, a, p)

In [30]:
g_evil = p

# Bob's private key is 'b'
b = np.random.randint(0, 2**31) % p
# Bob's public key is 'B'.  He computed this using g_evil -- and shares it with Alice.
B = pow(g_evil, b, p)

---
This seems to be the same situation.  If Mallory only manipulates the g used by one side, that results in a public key of ```B = 0``` for Bob, but no change for Alice.  If, on the other hand, Mallory can manipulate ```g``` before Alice generates her public key, they'll both generate a public key of zero, resulting in matching shared secrets and AES keys.

---

<div class="alert alert-block alert-info">

Do the MITM attack again, but play with "g". What happens with:

```g = 1```
    
```g = p```
    
**```g = p - 1```**
    
</div>

In [25]:
p = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff
g = 2

In [32]:
# Alice's private key is 'a'
a = np.random.randint(0, 2**31) % p

# Alice's public key is 'A'
A = pow(g, a, p)

In [33]:
g_evil = p - 1

# Bob's private key is 'b'
b = np.random.randint(0, 2**31) % p
# Bob's public key is 'B'.  He computed this using g_evil -- and shares it with Alice.
B = pow(g_evil, b, p)

In [43]:
sB = pow(B, a, p)

---
Again, same situation.  The difference here is that because  the shared secret computed by Alice will always = Bob's public key (= p-1). Same observation as for the other two cases.

---

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)